Primality Test
===============

It is usually hard to determine whether a given natural number is prime or not. The naive way is to use brute force to check if it has any prime factor other than itself. We have studied another algorithm, namely the Miller-Rabin Test, which is claimed to be less expenssive while still does the job.

The goal of this lab/project is to explore and implement the Miller-Rabin algorithm step-by-step in Python and analyse why it is faster.

In addition, there will also be an implementation of the AKS algorithm, but I will omit the details.

I hope we can get an idea of how a cryptography algorithm is implemented and what the potential issues could be in general.


# Brute-force


Let $n>1$ and we want to know if $n$ is prime. In practice, we can iterate through $2$ to $\lceil\sqrt{n}\,\rceil$ to see if it devides $n$. If any of these numbers divide $n$, then we draw the conclusion that $n$ is not prime.

We can assume that to determine whether $n\equiv0$ (mod $k$) is an easy task for the machine. So in the worst cases when $n$ is a prime number, the machine will have to do $\lceil\sqrt{n}\,\rceil - 1$ calculations, which is comparable to $n^{1/2}$.

Thus we say the (worst case) time complexity of this algorithm is $O(n^{1/2})$. (The space complexity is constant or $O(1)$ since we are not creating extra running variables.)



Let's import some packages first.

* `random` help us take random integers in a certain range or choose random element in a set.
* the functions in `math` are self-explanatory.
* `numpy` enables us to vectorize objects and thus do parallel computations.
* `time` help us know how long a piece of code runs so we can compare different algorithms.
* We use `decimal` in AKS to avoid loss of precision when doing logrithm or roots of large integers.

Run the following block by clicking on the upper-left corner of the block, or run it by click anywhere of the block and then apply `Ctrl+Enter`.

In [ ]:
from random import seed, randint
from math import log, gcd, sqrt
import numpy as np
import time
from decimal import getcontext, Decimal, ROUND_UP, ROUND_DOWN

Now let's implement the brute-force algorithm.

In [ ]:
def is_prime(n):
  if n <= 1:
    return False
  for i in range(2, n):
    if n % i == 0:
      return False
    if i * i >= n:
      return True
  return True

We know $n = 10^{17}-3$ is a large prime number. Let's see how long `is_prime(n)` will run.

In [ ]:
n = 10 ** 17 - 3
start = time.time()
ans = is_prime(n)
end = time.time()
print(ans)
print("Runtime: ", end-start, " seconds")

Typically, it takes about one minute to run this out if you are running it on Google Colab. (It might be a little longer if you are using SageMath on CoCalc.)

Therefore, we expect to have a better algorithm to solve it a little faster. Otherwise, we will not be able to apply our cryptography theory to larger integers. The Miller-Rabin Algorithm and the AKS Algorithm are both candidates.


# The Miller-Rabin Test

The Miller-Rabin Test is an algorithm that tells if a natural number is **not** a prime and is a candidate for our purpose.

Recall that if the algorithm returns `False`, we can declare that the input is not prime. While if the algorithm returns `True`, it only means that the input is "likely" a prime.

## Implementing the Miller-Rabin Algorithm

According to the pseudocode from lecture note, we can directly write the code. The input is a pair $(n,k)$ where $n$ is the positive integer that we want to determine the primality and the parameter $k$ is the number of tests to run.

Please refer to the textbook, the lecture note, or the wikipedia page [Miller-Rabin primality test](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test) if you are not familiar with the algorithm.

In [ ]:
def miller_rabin_v0(n, k):
  # some trivial cases
  if n == 1:
    return False
  if n == 2:
    return True

  # compute s and m
  s = 0
  m = n - 1
  while m % 2 == 0:
    m //= 2
    s += 1

  # determine if n is like a prime
  like_prime = True
  for _ in range(k):
    a = randint(2, n-1)
    b = a ** m % n
    if b != 1:
      like_prime = False
      for r in range(s):
        if a ** (2 ** r * m) % n == n - 1:
          like_prime = True
          break
      # if any a witnesses n, then n cannot be a prime
      if not like_prime:
        break

  # if none of these randomly chosen a's witness n, then the
  # likelyhood of n being non-prime is less than 4^{-k}.
  return like_prime

Let's see if $n$ *looks like* a prime for $n=1, 2, \ldots, 20$.

In [ ]:
#seed(10)
for i in range(1, 21):
  print(f"{i}:{miller_rabin_v0(i,1)}", end="; ")

You may or may not find that it makes some mistakes. For example, if you set the random seed as $10$ by uncommenting `seed(10)`, you will see that it says $9$ is likely a prime, which is false.

This is very natural since we only take one test for each number. In this case, the algorithm can only guarantee that the likelyhood of $n$ being a prime is no greater than $\cfrac{1}{4}=25\%$, which is pretty high. If you run the previous block several times, the accuracy can be much higher.

**If you are doing so, remember to comment out or delete `seed(10)`, otherwise you will always get the same results.**

Alternatively, we can run more tests each time by setting a larger $k$.

In [ ]:
seed(10)
k = 2
for i in range(1, 21):
  print(f"{i}:{miller_rabin_v0(i,k)}", end="; ")

Even if we only increase $k$ by one, the possibility of encountering a Fermat Liar will decrease significantly. In our case, the false positive rate will be less than $\cfrac{1}{4^2}=6.25\%$.

Empirically, the false positive rate could be significantly lower than $\cfrac{1}{4^k}$. We will see this later.

## Improvement

Our implementation of the Miller-Robin algorithm is, unfortunetely, defective.

Let's see the example $n = 2312311$, which we know is a prime.

Let's compare the runtime of `miller_rabin_v0(2312311)` and `is_prime(2312311)`.

In [ ]:
start = time.time()
miller_rabin_v0(2312311, 1)
end = time.time()
print("MRv0 runtime: ", end-start, " seconds")
start = time.time()
is_prime(2312311)
end = time.time()
print("Brute-force runtime: ", end-start, " seconds")

Suprisingly, our Miller-Robin algorithm is even slowlier than brute-force for this not-so-large prime number, even if we only run one test ($k=1$).

This is bad because it intimates that it may take even longer to run `miller_robin_v0(10**17-3)`.

### Troubleshooting

In the Miller-Robin algorithm, essentially we did three things:

* Find $s$ and $m$.
* Compute $a^m$ mod $n$.
* Compute $a^{2^rm}$ mod $n$, for $r=0, 1, \ldots, n-1$.

The first task is easy for the machine: the worst time complexity is $O(\log_2 n)$. So the real problem is how we compute the powers.

Recall that for fixed $a$ and $m$, we can do $a^{2^rm} = (a^{2^{r-1}m})^2$. So, if we know $a^m$ mod $n$, we can inductively get $a^{2^rm}$ for all $r$.

That is, instead of computing $a^{2^rm}$ for each $r$ separately, we can make use of $a^{2^{r-1}m}$ to reduce the computational cost. This idea is known as *dynamic programming*.


In fact, we have already used this idea in our homework.

> How to compute $5^{93}$ mod $101$ without doing too much computation?

We wrote $93$ as a sum of powers of $2$ and we computed $5^{2^r}$ by taking squares and modulo the squares by $101$.



The following function `modular_power` is an implementation using this idea.

In [ ]:
def modular_power(base, power, modular):
  assert power >= 0, "Power must be non-negative."

  if power == 0:
    return 1

  # transfer the power into binary
  power_binary = [int(i) for i in list('{0:0b}'.format(power))]

  # find the highest power of 2 that we need to compute
  highest = len(power_binary) - 1

  # inductively calculate the powers
  power_lst = [base % modular]
  for _ in range(highest):
    power_lst.insert(0, power_lst[0] * power_lst[0] % modular)

  # combine the powers
  ans = 1
  for i in range(highest+1):
    if power_binary[i]:
      ans = ans * power_lst[i] % modular

  return ans

Another advantage of this implementation is that it avoids `int` type overflow when doing exponentials. This shouldn't be an issue in Python3, but we do want to avoid it as much as we can since dynamically enlarging the upperbounds of the data is also time-consuming. Another reason is that Numpy does not automatically enlarge the upperbounds and we will use Numpy.

### Improved Miller-Rabin

With `modular_power` in hand, we can modify our function.

In [ ]:
def miller_rabin_v1(n, k):
  # same as MRv0
  if n == 1:
    return False
  if n == 2:
    return True

  # same as MRv0
  s = 0
  m = n - 1
  while m % 2 == 0:
    m //= 2
    s += 1

  like_prime = True
  for _ in range(k):
    a = randint(2, n-1)

    # invoke modular_power()
    b = modular_power(a, m, n)

    if b != 1:
      like_prime = False

      # since we already know b, we just need to do
      # b**(2**r) inductively.
      for _ in range(s):
        if b == n - 1:
          like_prime = True
          break
        else:
          b = b * b % n

      # same as MRv0
      if not like_prime:
        break

  # if none of these randomly chosen a's witness n, then the
  # likelyhood of n being non-prime is less than 4^{-k}.
  return like_prime

Now let's see its performance for the input $n = 2312311$.

In [ ]:
start = time.time()
ans = miller_rabin_v1(2312311, 1)
end = time.time()
print("MRv1 runtime: ", end - start)

Incredible improvement, isn't it!
------------------------------------

## Determinist Miller-Rabin Test

Recall that there is a Determinist Miller-Rabin Test:
> Assuming *General Riemann Hypothesis*(GRH), if $n$ is an odd composite, then there is a witness $a$ for $n$ such that $$a \leq \lfloor2(\ln(n))^2\rfloor.$$

For simplicity, we implement the function `miller_rabin()` by combining the Determinist Miller-Rabin Test into the ordinary Miller-Rabin Test.

`miller_rabin(n, k=-1)` has two parameters:

* `n` is the positive integer we want to check primality of.
* `k` is the number of tests to run if it is not $-1$. If $k=1$, it calls the Determinist Miller-Robin. `k` has a default value $1$.

In [ ]:
def miller_rabin(n, k=-1):
  assert k >= -1, "Invalid k."
  if n == 1:
    return False
  if n == 2:
    return True
  if n % 2 == 0:
    return False
  s = 0
  m = n - 1
  while m % 2 == 0:
    m //= 2
    s += 1
  like_prime = True
  if k != -1:
    for _ in range(k):
      like_prime = True
      a = randint(2, n-1)
      b = modular_power(a, m, n)
      if b != 1:
        like_prime = False
        for _ in range(s):
          if b == n - 1:
            like_prime = True
            break
          else:
            b = b * b % n
        if not like_prime:
          return False
  else:
    for a in range(2, min(int(2*log(n)*log(n))+1, n-1)):
      like_prime = True
      b = modular_power(a, m, n)
      if b != 1:
        like_prime = False
        for _ in range(s):
          if b == n - 1:
            like_prime = True
            break
          else:
            b = b * b % n
        if not like_prime:
          return False
  return like_prime

vmiller_rabin = np.vectorize(miller_rabin)

We can do some trivial tests.

In [ ]:
print(miller_rabin(11))
print(miller_rabin(15, 1))
print(miller_rabin(10**17+5))
print(miller_rabin(29, 3))

So this is actually a feasible implementation of Miller-Rabin!

## Algorithm Analysis

### Complexity of finding $s$ and $m$:

We quotient $n$ down by $2$ each time and repeat at most $\log_2n$ times. So the time complexity is $O(\log n)$. We only created two variables $s$ and $m$, so the space complexity is constant.





### Complexity of modular power $a^N$ (mod $b$):

Recall that this depends on $n$, the number of digits in the binary expression of $N$. Approximately, we have $n ≈ \log N$ and it is independent of $a$ or $b$. Also, we created an array to store the values of $a, a^2, a^4, \ldots, a^{2^n}$.

So the time complexity and space complexity are both $O(\log N)$.

### Complexity of a single test:

We randomly pick $a$ and run `modular_power(a, m, n)` once to compute $b = a^m$, which consumes $O(\log m)$. Then we inductively compute $b^2, b^4, \ldots, b^{2^{s}}$, which takes $s$ steps.

So the time complexity of running `miller_rabin(n, 1)` is
$$O(\log n) + O(\log m) + O(s).$$

Note that we always have $s≈\log n$. In worst cases, we have $m = (n-1)/2$, so $m≈n$. Therefore the time complexity is $O(\log n)$.

Extra memory is only required when running `modular_power(a, m, n)`. So the total space complexity is $O(\log m)$. So in the worst cases, the space complexity is $O(\log n)$.

### Complexity of `miller_rabin(n, k)`:

If we run $k$ tests, the only difference will be the last step, since we repeat the last step for $k$ times. So the time complexity is
$$O(\log n) + O(\log m) + O(ks) = O(k\log n).$$

### Complexity of DMR (`miller_rabin(n)`):

The DMR repeat the last step for at most $\lfloor2\cdot(\log n)^2\rfloor$ times. So it is similar to the case when $k = \lfloor2\cdot(\log n)^2\rfloor$. Therefore, the time complexity is $O(\log^3 n)$.

The space complexity is still $O(\log n)$.

## On the accuracy of (non-determinist) Miller-Rabin

Recall that if `miller_rabin(n, k)` returns `True`, it means that $n$ is very likely a prime number, while the possibility that it is not prime is less than $4^{-k}$. We have seen before that when we set $k=1$, it could make mistakes. Here's another example.

### Example.

Let $N=233333$. One can use either `is_prime(233333)` or `miller_rabin(233333)` to check that it is **not a prime number**. However, `miller_rabin(233333, 1)` may say it looks like a prime number.

In [ ]:
seed(574)
miller_rabin(233333, 1)

This is because the function will always pick $a = 81585$ (as we set the random seed) and this happens to be a Fermat Liar for $N = 233333$.

Without setting a random seed, the result might be correct (maybe still wrong).

In [ ]:
miller_rabin(233333, 1)

Let's run `miller_rabin(233333, 1)` for multiple times and see how many mistakes it will make.

We take one million samples.

In [ ]:
N = 233333
samples = 1000000

In [ ]:
array_N = np.ones((1, samples), dtype=int) * N

seed(10)
print(f"Sampling {samples} times, one test each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 1).astype(int)))

seed(10)
print(f"Sampling {samples} times, two tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 2).astype(int)))

seed(10)
print(f"Sampling {samples} times, three tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 3).astype(int)))

It means that for $n=233333$, the false positive rate of MR single-test is approximately $3046/1000000 = 0.346\%$ and the false positive rate of MR two-test is approximately $7\times10^{-6}$.

If you set `samples = 10000000` (ten million), it will take approximately seven minutes and give the following result:



```
Sampling 10000000 times, one test each time:
Number of mistakes =  31009
Sampling 10000000 times, two tests each time:
Number of mistakes =  91
Sampling 10000000 times, three tests each time:
Number of mistakes =  1
```



Loosely speaking, `miller_robin(233333, 3)` makes one mistake per ten million calls.

### Example.

For different $N$, the accuracy can be different.

In [ ]:
N = 233133
samples = 1000000

In [ ]:
is_prime(N)

In [ ]:
array_N = np.ones((1, samples), dtype=int) * N

seed(10)
print(f"Sampling {samples} times, one test each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 1).astype(int)))

seed(10)
print(f"Sampling {samples} times, two tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 2).astype(int)))

You can see that even the single-test MR yields a very low false positive rate.

So the accuracy of Non-determinist MR `miller_rabin(n, k)` depends on both $n$ and $k$. Let's see the following example.

### Example.

Let $N_1 = 32767 = 2^{15}-1$ and $N_2 = 32769 = 2^{15}+1$.

In [ ]:
N1 = 32767
N2 = 32769
samples = 1000000

In [ ]:
print(is_prime(N1))
print(is_prime(N2))

So neither of them are prime.

In [ ]:
N = N1
array_N = np.ones((1, samples), dtype=int) * N

seed(10)
print(f"Sampling {samples} times, one test each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 1).astype(int)))

seed(10)
print(f"Sampling {samples} times, two tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 2).astype(int)))

seed(10)
print(f"Sampling {samples} times, three tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 3).astype(int)))

In [ ]:
N = N2
array_N = np.ones((1, samples), dtype=int) * N

seed(10)
print(f"Sampling {samples} times, one test each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 1).astype(int)))

seed(10)
print(f"Sampling {samples} times, two tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 2).astype(int)))

seed(10)
print(f"Sampling {samples} times, three tests each time:")
print("Number of mistakes = ", np.sum(vmiller_rabin(array_N, 3).astype(int)))

Can you figure out why there is such a big difference?

#### Hint:

Consider the relation between $s$ and number of Fermat Liars.

# Comparing the Runtimes

Let's compare the actual runtime of brute-force, Miller-Rabin with single test, and Determinist Miller-Rabin on a relatively large prime number.

## Example.

$N = 11111117$.

In [ ]:
N = 11111117

start = time.time()
ans = is_prime(N)
end = time.time()
print("Brute-force runtime: ", end-start)
print("Brute-force result: ", ans)

start = time.time()
ans = miller_rabin(N, 1)
end = time.time()
print("MR single-test runtime: ", end-start)
print("MR single-test result: ", ans)

start = time.time()
ans = miller_rabin(N)
end = time.time()
print("DMR runtime: ", end-start)
print("DMR result: ", ans)


In this example, we can see:
$$\text{Speed of DMR} < \text{Speed of Brute-force} < \text{Speed of MR Single-test}.$$

## Example.

$N = 10^{17}-3$. This is the large number on which we found `is_prime(N)` was not satisfactory.



In [ ]:
N = 10 ** 17 - 3

start = time.time()
ans = is_prime(N)
end = time.time()
print("Brute-force runtime: ", end-start)
print("Brute-force result: ", ans)

start = time.time()
ans = miller_rabin(N, 1)
end = time.time()
print("MR single-test runtime: ", end-start)
print("MR single-test result: ", ans)

start = time.time()
ans = miller_rabin(N)
end = time.time()
print("DMR runtime: ", end-start)
print("DMR result: ", ans)

Now we have
$$\text{Speed of Brute-force} \ll \text{Speed of DMR} < \text{Speed of MR Single-test}.$$

Additionally, as we have seen earlier, the MR Single-test is accurate enough in most cases. Now that MR Single-test can be $1000$ times faster than DMR and $200,000$ times faster than Brute-force for large numbers, it is actually a very good algorithm for the purpose of crypto.

The following code block compares the performances for $n$ in different magnitude. We can see that DMR starts to come out on top of brute-force when $n ≈ 10^{10}$.

*Typically, it takes about 80 seconds to run this block.*

In [ ]:
disagree = 0
K = 100
seed(1)
for i in range(5, 16):
  print(f"Running {K} samples around 10^{i}:")
  samples = []
  for _ in range(K):
    samples.append(randint(10**i // 2, 10**i * 5))
  start = time.time()
  for j in range(K):
    is_prime(samples[j])
  end = time.time()
  print(f"Brute-Force: {end - start} seconds", end="; ")
  start = time.time()
  for j in range(K):
    miller_rabin(samples[j], 1)
  end = time.time()
  print(f"MR Single-test: {end - start} seconds", end="; ")
  start = time.time()
  for j in range(K):
    miller_rabin(samples[j])
  end = time.time()
  print(f"DMR: {end - start} seconds")

# The Agrawal-Kayal-Saxena Primality Test

Here is an (toy) implementation of the *Agrawal-Kayal-Saxena(AKS) Algorithm*.

From [this wikipedia page](https://en.wikipedia.org/wiki/AKS_primality_test) and AKS' original paper, the algorithm is as follow:






> Input: integer n > 1.
> 1. If ($n = ab$ for $a \in N$ and $b > 1$), output COMPOSITE.
> 2. Find the smallest $r$ such that the order of $r$ in $\mathbb{Z}/n\mathbb{Z}$ is greater than $\ln^2n$. If $\gcd(r,n) > 1$, output COMPOSITE.
> 3. If $a$ divides $n$ for some $2 \leq a \leq \min(r,n-1)$, output COMPOSITE.
> 4. If $n \leq r$, output PRIME.
> 5. For $a = 1$ to $\lfloor\sqrt{\phi(r)}\ln n\rfloor$ and for $c$ from $0$ to $r$, if $h_a(c)\not\equiv0$ (mod $n$), return COMPOSITE.
> 6. Output PRIME.





I will omit most of the details.

In [ ]:

# It is a little bit tricky to determine whether n is a perfect power.
def is_perfect_power(n):
  digits = len(str(n))
  getcontext().prec = 2 * digits
  n = Decimal(n)
  for k in range(2, int(n.ln()/Decimal(2).ln()) + 1):
    a = n ** (Decimal(1.0) / Decimal(k))
    if a.quantize(Decimal('1.'), rounding=ROUND_UP) - a < min(Decimal(1e-6), Decimal(1)/n) or\
      a - a.quantize(Decimal('1.'), rounding=ROUND_DOWN) < min(Decimal(1e-6), Decimal(1)/n):
      return True
  return False

# We implement the Euler totient function by brute-force since it is only
# applied to r, which is always small in AKS.
def euler_totient(n):
  ans = 0
  for k in range(1, n+1):
    if gcd(n, k) == 1:
      ans += 1
  return ans

def aks(n):
  if n == 1:
    return False
  if n == 2:
    return True

  # Step 1: if n is a perfect power, then it is composite.
  if is_perfect_power(n):
    return False

  digits = len(str(n))
  getcontext().prec = 2 * digits
  n = Decimal(n)

  # Step 2: find r
  log_of_n = n.ln()
  r = 2
  while r < n:
    r_power = Decimal(r)
    order_is_large = True
    for _ in range(2, int(log_of_n*log_of_n)+1):
      r_power = r_power * Decimal(r) % n
      if r_power == 1:
        order_is_large = False
        break
    if order_is_large:
      break
    else:
      r += 1

  # Step 2: check if r and n are coprime
  n = int(n)
  if gcd(r, n) != 1:
    return False

  # Step 3: check condition
  for i in range(2, min(r, n-1) + 1):
    if n % i == 0:
      return False

  # Step 4: check condition
  if r == n:
    return True

  # Step 5: compute and check h_a(c)
  for a in range(1, int(Decimal(sqrt(euler_totient(r))) * log_of_n) + 1):
    powers = [1]
    for k in range(1, n):
      powers.append(powers[-1] * a % n)
    coeff = [(modular_power(a, n, n) - a) % n]
    prev = 1
    for k in range(1, n):
      prev = prev * (n-k+1) // k
      coeff.append(powers[k] * (prev % n) % n)
    for c in range(r+1):
      hac = 0
      for k in range(n):
        if k % r == c:
          hac = (hac + coeff[k]) % n
      if hac != 0:
        return False

  # Step 6: draw conclution
  return True




Let's see how it works for $n$ from $1$ to $20$.

In [ ]:
for i in range(1, 21):
  print(f"{i}:{aks(i)}", end="; ")

If we are not confident enough with the implementation, we can run `is_prime(N)` and `aks(N)` for $N=1$ to $2000$ and see how many of the results disagree.

In [ ]:
disagree = 0
for i in range(1, 2001):
  if is_prime(i) != aks(i):
    disagree += 1
print(disagree, " mistakes out of ", 2000, " tests.")

So we are good.

However, you may have noticed that it is surprisingly slow. Let's try a larger number and compare how different algorithm runs on it.

In [ ]:
N = 9997

start = time.time()
ans = is_prime(N)
end = time.time()
print("Brute-force runtime: ", end-start)
print("Brute-force result: ", ans)

start = time.time()
ans = miller_rabin(N, 1)
end = time.time()
print("MR single test runtime: ", end-start)
print("MR single test result: ", ans)

start = time.time()
ans = miller_rabin(N)
end = time.time()
print("DMR runtime: ", end-start)
print("DMR result: ", ans)

start = time.time()
ans = aks(N)
end = time.time()
print("AKS runtime: ", end-start)
print("AKS result: ", ans)


It seems our implementation of AKS is the slowest one comparing to MR, DMR and even the brute-force!

In fact, if we run `aks(N)` for $N≈10^6$, it will take approximately ten seconds.

If we run `aks(N)` for $N≈10^7$, it will take more than twenty minutes.

## A Drawback of the Implementation

Theoretically, the time complexity of AKS is a polynomial of the number of digits of the input integer, namely $O(\log^k n)$ for some natural number $k$.

If this is the case, it should at least defeat the brute-force, since $\log^kn = O(n^{1/2})$ but $n^{1/2}\neq O(\log^k n)$ for all $k > 0$.

Here is the problem: when $n$ is large, it is extremely hard to find the coefficients of
$$(x+a)^n-(x^n+a)$$
as a polynomial in $\mathbb{Z}/n\mathbb{Z}$.



For example, let $n=10000$. Then the coefficient of the degree-$5000$ term will be
$$C_{10000}^{5000} \cdot a^{5000} \;\mathrm{mod}\; 10000,$$
where $C_n^k$ is the combinatorial number
$$\cfrac{n!}{(n-k)!k!}.$$

However, the combinatorial number $C_{10000}^{5000}$ has $3009$ digits. So it is super hard to compute $C_{10000}^{5000}$ mod $10000$.


**The bottleneck is:** how to compute $C_n^k$ mod $n$, suppose we already know $C_n^{k-1}$ mod $n$?

It seems $C_n^k$ mod $n$ does not quite depend on the previous term, e.g.
$$C_8^1\equiv0,\; C_8^2\equiv4,\; C_8^3\equiv0,\; C_8^4\equiv6,\; C_8^5\equiv0,\; C_8^6\equiv4,\; C_8^7\equiv0.$$

So we cannot use inductions or dynamic programming to simplify it.

Comparing to the Miller-Rabin or Determinist Miller-Rabin, an advantage of the AKS algorithm is that **it is more reliable**, in the sense that it does not depend on any unproven hypothesis and is hundred percent correct if implemented correctly.
-----------

However, our implementation is too slow to overtake the most reliable one -- the brute-force. That is why I say it is just a toy implementation.
-----------

Personally, I recommend `miller_rabin(n)` if you just want to grab one to use as a calculator.
-----------

Anyway, if you know a better way to calculate the coefficients or you find a better implementation of the AKS algorithm, please let me know.
-----------------------

Thanks for reading this notebook! Feel free to change the parameters and explore by yourself!
=============